In [1]:
from acfunctions import *
import numpy as np
import pandas as pd
from solartwins import *
import pickle
import starspot as ss
import starspot.rotation_tools as rt
import lightkurve as lk
from lightkurve import search_targetpixelfile, search_lightcurvefile
import exoplanet
import eleanor
from scipy import interpolate
from tess_kep_funct import *
from tqdm import tqdm
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import warnings
warnings.filterwarnings("ignore")

/Users/ilanadoran/opt/anaconda3/lib/python3.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

# key 

short_tess_period = periods calculated using starspot's pdm period function without linear interpolation

short_periods = calculated mcquillan periods under 13 days 

tics_short_periods = tic ids of the mcquillan-determined short periods

ls_periods_interp = calculated lombs-scargle tess periods from starspot with linear interpolation

tess_periods_interp = periods calculated using pdm function and linear interpolation

differences_mcq_pdm = differences between non-linear interp PDM and McQ tess periods

differences_exo_acf = differences between Exoplanet ACF with interp and McQ tess periods

differences_mcq_ls = differences between L-S with interp and McQ tess periods

differences_ls_pdm = differences between Lomb-Scargle with interp and PDM calculated tess period without interp

differences_tess = differences between linear PDM calculations and McQ tess periods


# importing data from TESS_Kep

In [3]:
#dataset with tess vs kepler IDs
tess_kep = pd.read_csv('data_summary.csv')
tess_short = tess_kep[tess_kep.tess_ffi == True]
cat = pd.read_csv('tics_kics_mcq_rot.txt', names=['tic_id', 'kic_id', 'prot'], delim_whitespace=True)

In [4]:
#importing McQ periods
headers = 'TIC ID', 'KIC ID', 'McQ Periods';  

with open("tics_kics_mcq_rot.txt") as file:
    rot_table = [line.split() for line in file.read().splitlines()]

widths = [max(len(value) for value in col)
for col in zip(*(rot_table + [headers]))]

formatting = '{:{widths[0]}}  {:{widths[1]}}  {:{widths[2]}}'
formatting.format(*headers, widths=widths)
for tics_kics_mcq in rot_table:
    formatting.format(*tics_kics_mcq, widths=widths)

In [5]:
# converting txt file data to arrays
tic_ids = []
mcq_periods = []

for index, row in enumerate(rot_table):
    tic_ids.append(int(rot_table[index][0]))
    mcq_periods.append(float(rot_table[index][2]))

In [6]:
# selecting shortest periods only (<13 days, half a tess cycle)
short_periods = []
tics_short_periods = []

for index, period in enumerate(mcq_periods):
    if period < 13: 
        short_periods.append(period)
        tics_short_periods.append(tic_ids[index])
        
#limiting to the first 60
short_periods = short_periods[0:]
tics_short_periods = tics_short_periods[0:70]

In [7]:
with open('short_periods.data', 'wb') as filehandle:
    pickle.dump(short_periods, filehandle)

In [8]:
with open('tess_period.data', 'rb') as filehandle:
    short_tess_period = pickle.load(filehandle) 

In [9]:
output_notebook()

Loading BokehJS ...

# calculating tess short periods using pdm and lomb-scargle with linear interpolation

In [10]:
tess_periods = []
ls_periods = []
exo_periods = []

for index, value in (enumerate(tics_short_periods[0:])):
    period_tess = eleanor_plots(value, index)
    tess_periods.append(period_tess[1])
    ls_periods.append(period_tess[0])
    exo_periods.append(period_tess[2])

No eleanor postcard has been made for your target (yet). Using TessCut instead.




100%|██████████| 1000/1000 [00:00<00:00, 1649.51it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1000/1000 [00:00<00:00, 1855.31it/s]


fitting to a limited range
No eleanor postcard has been made for your target (yet). Using TessCut instead.


...

100%|██████████| 1000/1000 [00:00<00:00, 1701.83it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.
Inflating...


100%|██████████| 1000/1000 [00:00<00:00, 1802.07it/s]


fitting to a limited range
No eleanor postcard has been made for your target (yet). Using TessCut instead.
Inflating...


100%|██████████| 1000/1000 [00:00<00:00, 1318.10it/s]


In [11]:
len(short_periods)

317

tess_periods_interp = tess_periods

ls_periods_interp = ls_periods

with open('tess_periods_interp.data', 'wb') as filehandle:
    pickle.dump(tess_periods_interp, filehandle)

with open('ls_periods_interp.data', 'wb') as filehandle:
    pickle.dump(ls_periods_interp, filehandle)

In [11]:
with open('tess_periods_interp.data', 'rb') as filehandle:
    tess_periods_interp = pickle.load(filehandle) 

In [12]:
with open('ls_periods_interp.data', 'rb') as filehandle:
    ls_period_interp = pickle.load(filehandle) 

#calculating tess periods from exoplanet
exo_ls = [] #lombs-scargle
exo_acf = [] #acf

for each, star in enumerate(tics_short_periods):
    star_el = eleanor.Source(tic=star)
    data = eleanor.TargetData(star_el, do_psf=True, do_pca=True)
    q = data.quality == 0
    
    time = data.time[q]    
    flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
    f = interpolate.interp1d(time, flux)
    xnew = np.arange(time[0], time[-1], 0.1)
    norm_flux = f(xnew) 
    flux_err = np.sqrt(norm_flux)

    exo = exoplanet.lomb_scargle_estimator(xnew, norm_flux, yerr=flux_err, min_period= 0.1, max_peaks=2)
    ex_1 = exoplanet.autocorr_estimator(xnew, norm_flux, yerr=flux_err, oversample=2.0, smooth=2.0, )
    
    if len(exo['peaks']) > 0: 
        exo_ls.append(exo['peaks'][0]['period'])  
    else: 
        exo_ls.append(0)
        
    if len(ex_1['peaks']) > 0: 
        exo_acf.append(ex_1['peaks'][0]['period'])
    else:
        exo_acf.append(0)  

with open('exo_ls.data', 'wb') as filehandle:
    pickle.dump(exo_ls, filehandle)

with open('exo_acf.data', 'wb') as filehandle:
    pickle.dump(exo_acf, filehandle)

In [13]:
with open('exo_ls.data', 'rb') as filehandle:
    exo_ls = pickle.load(filehandle) 

with open('exo_acf.data', 'rb') as filehandle:
    exo_acf = pickle.load(filehandle) 

In [14]:
x =  short_periods
y = short_tess_period

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = short_tess_period, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

pdm_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="PDM Period vs McQuillan Periods",
   x_axis_label='McQuillan Period', y_axis_label='TESS PDM Period', plot_width=900)

pdm_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
pdm_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
pdm_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
pdm_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'rosybrown', line_dash="dotted")

pdm_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_plot)

In [15]:
#finding differences between PDM and McQ tess periods
differences_mcq_pdm = []

for indx, period in enumerate(short_tess_period):
    diff = np.abs(period - short_periods[indx])
    differences_mcq_pdm.append(diff)

In [16]:
x =  short_periods
y = differences_mcq_pdm

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_mcq_pdm, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

pdm_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 12], title="Differences Between McQ and PDM TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

pdm_diff.line(x, x_0, name = "y=0", color = 'bisque', line_dash="4 4", legend = "y=0", line_width = 2)
pdm_diff.circle('x', 'y', source = source, color="mediumseagreen", line_color=None)


pdm_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_diff)

In [17]:
x =  short_periods
y = exo_acf

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = exo_acf, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

pdm_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Exoplanet ACF vs McQuillan Periods",
   x_axis_label='McQuillan Period', y_axis_label='Exoplanet ACF Period', plot_width=900)

pdm_plot.line(x, x, name = "y=x", color = 'lightseagreen', line_dash="4 4", legend = "y=x", line_width = 2)
pdm_plot.circle('x', 'y', source = source, color="darkgreen", line_color=None)
pdm_plot.line(x, x2, legend="y=2x", line_width=1, color = 'mediumpurple', line_dash="dotted")
pdm_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'skyblue', line_dash="dotted")

pdm_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_plot)

In [18]:
#finding differences between Exoplanet ACF and McQ tess periods
differences_exo_acf = []

for indx, period in enumerate(exo_acf):
    diff = np.abs(period - short_periods[indx])
    differences_exo_acf.append(diff)

In [19]:
x =  short_periods
y = exo_acf

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = exo_acf, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

ls_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 9], title="Differences Between McQ and Exoplanet ACF TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

ls_diff.line(x, x_0, name = "y=0", color = 'lightgreen', line_dash="4 4", legend = "y=0", line_width = 2)
ls_diff.circle('x', 'y', source = source, color="indigo", line_color=None)


ls_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_diff)

In [20]:
x =  short_periods
y = ls_period_interp

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = ls_period_interp, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

ls_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Lomb-Scargle with Interpolation vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

ls_plot.line(x, x, name = "y=x", color = 'palegreen', line_dash="4 4", legend = "y=x", line_width = 2)
ls_plot.circle('x', 'y', source = source, color="darkcyan", line_color=None)
ls_plot.line(x, x2, legend="y=2x", line_width=1, color = 'dimgray', line_dash="dotted")
ls_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'sienna', line_dash="dotted")

ls_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_plot)

In [21]:
#finding differences between L-S and McQ tess periods
differences_mcq_ls = []

for indx, period in enumerate(ls_period_interp):
    diff = np.abs(period - short_periods[indx])
    differences_mcq_ls.append(diff)

In [22]:
x =  short_periods
y = differences_mcq_ls

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_mcq_ls, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

ls_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 12], title="Differences Between McQ and L-S with Interpolation TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

ls_diff.line(x, x_0, name = "y=0", color = 'tomato', line_dash="4 4", legend = "y=0", line_width = 2)
ls_diff.circle('x', 'y', source = source, color="black", line_color=None)


ls_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_diff)

In [23]:
#finding differences between Lomb-Scargle and PDM calculated tess period
differences_ls_pdm = []

for indx, period in enumerate(short_tess_period):
    tess_diff = np.abs(period - ls_period_interp[indx])
    differences_ls_pdm.append(tess_diff)

In [24]:
x =  short_periods
y = differences_ls_pdm

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_ls_pdm, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

diff_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between L-S and PDM TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

diff_plot.line(x, x_0, name = "y=0", color = 'lightpink', line_dash="4 4", legend = "y=0", line_width = 2)
diff_plot.circle('x', 'y', source = source, color="darkred", line_color=None)


diff_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(diff_plot)

with open('ls_period.data', 'rb') as filehandle:
    ls_period = pickle.load(filehandle) 

In [25]:
with open('ls_period_no_interp.data', 'rb') as filehandle:
    ls_period_no_interp = pickle.load(filehandle) 

#finding differences between Lomb-Scargle with and without interpolation
differences_ls = []

for indx, period in enumerate(ls_period_interp):
    ls_diff = np.abs(period - ls_period_no_interp[indx])
    differences_ls.append(ls_diff)

x =  short_periods
y = differences_ls

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_ls, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

ls_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between L-S with and without Linear Interpolation",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between LS Calculations', plot_width=900)

ls_plot.line(x, x_0, name = "y=0", color = 'plum', line_dash="4 4", legend = "y=0", line_width = 2)
ls_plot.circle('x', 'y', source = source, color="firebrick", line_color=None)


ls_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(ls_plot)

In [27]:
x =  short_periods
y = tess_periods_interp

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = tess_periods_interp, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

tess_interp_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="PDM with Interpolation vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

tess_interp_plot.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)
tess_interp_plot.circle('x', 'y', source = source, color= "purple", line_color=None)
tess_interp_plot.line(x, x2, legend="y=2x", line_width=1, color = 'orange', line_dash="dotted")
tess_interp_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'powderblue', line_dash="dotted")

tess_interp_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(tess_interp_plot)

In [28]:
#finding differences between Lomb-Scargle with and without interpolation
differences_tess = []

for indx, period in enumerate(tess_periods_interp):
    tess_diff = np.abs(period - short_periods[indx])
    differences_tess.append(tess_diff)

In [29]:
x =  short_periods
y = differences_tess

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_tess, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

tess_diff_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between PDM with and without Linear Interpolation",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between LS Calculations', plot_width=900)

tess_diff_plot.line(x, x_0, name = "y=0", color = 'powderblue', line_dash="4 4", legend = "y=0", line_width = 2)
tess_diff_plot.circle('x', 'y', source = source, color="firebrick", line_color=None)


tess_diff_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(tess_diff_plot)

In [30]:
#differences between non-linear interp PDM and McQ tess periods
np.average(differences_mcq_pdm)

3.323581450302761

In [31]:
#differences between Exoplanet ACF with interp and McQ tess periods
np.average(differences_exo_acf)

3.663442622951167

In [32]:
#differences between L-S with interp and McQ tess periods
np.average(differences_mcq_ls)

3.6170666909565936

In [33]:
#differences_tess
np.average(differences_tess)

3.2363680823680823

In [34]:
with open('ls_periods_resid.data', 'rb') as filehandle:
    ls_periods_resid = pickle.load(filehandle) 

In [35]:
with open('pdm_periods_resid.data', 'rb') as filehandle:
    pdm_periods_resid = pickle.load(filehandle) 

In [36]:
x =  short_periods[0:50]
y = ls_periods_resid

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = ls_periods_resid, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

ls_resid_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Lomb-Scargle with Trends Removed vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

ls_resid_plot.line(x, x, name = "y=x", color = 'powderblue', line_dash="4 4", legend = "y=x", line_width = 2)
ls_resid_plot.circle('x', 'y', source = source, color= "blue", line_color=None)
ls_resid_plot.line(x, x2, legend="y=2x", line_width=1, color = 'red', line_dash="dotted")
ls_resid_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'pink', line_dash="dotted")

ls_resid_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_resid_plot)

#differences between interpolations
difference_resid = []

for indx, ls in enumerate(ls_periods_resid):
    resid_diff = np.abs(ls - short_tess_period[indx])
    difference_resid.append(resid_diff)

x =  short_periods
y = difference_resid

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = difference_resid, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

resid_ls_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between L-S Residuals and McQ",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between LS Calculations', plot_width=900)

resid_ls_plot.line(x, x_0, name = "y=0", color = 'pink', line_dash="4 4", legend = "y=0", line_width = 2)
resid_ls_plot.circle('x', 'y', source = source, color="black", line_color=None)


resid_ls_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(resid_ls_plot)

#differences between interpolations
difference_ls_interp_resid = []

for ls, indx in enumerate(ls_periods_resid):
    ls_interp_diff = np.abs(ls - ls_period_interp[indx])
    difference_ls_interp_resid.append(ls_interp_diff)

x =  short_periods
y = difference_ls_interp_resid

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = difference_ls_interp_resid, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

ls_interps_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between L-S with and without Trend Removals",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between LS Calculations', plot_width=900)

ls_interps_plot.line(x, x_0, name = "y=0", color = 'powderblue', line_dash="4 4", legend = "y=0", line_width = 2)
ls_interps_plot.circle('x', 'y', source = source, color="navy", line_color=None)


ls_interps_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(ls_interps_plot)

In [42]:
x =  short_periods[0:50]
y = pdm_periods_resid

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = pdm_periods_resid, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

pdm_resid_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="PDM Lomb-Scargle vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

pdm_resid_plot.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)
pdm_resid_plot.circle('x', 'y', source = source, color= "purple", line_color=None)
pdm_resid_plot.line(x, x2, legend="y=2x", line_width=1, color = 'orange', line_dash="dotted")
pdm_resid_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'powderblue', line_dash="dotted")

pdm_resid_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_resid_plot)

#differences between interpolations
difference_pdm_interp_resid = []

for pdm in pdm_periods_resid:
    pdm_interp_diff = np.abs(pdm - tess_periods_interp[indx])
    difference_pdm_interp_resid.append(pdm_interp_diff)

np.average(difference_pdm_interp_resid)

x =  short_periods
y = difference_pdm_interp_resid

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = difference_pdm_interp_resid, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

tess_pdms_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between PDM with and without Trends Removed",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between LS Calculations', plot_width=900)

tess_pdms_plot.line(x, x_0, name = "y=0", color = 'powderblue', line_dash="4 4", legend = "y=0", line_width = 2)
tess_pdms_plot.circle('x', 'y', source = source, color="firebrick", line_color=None)


tess_pdms_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(tess_pdms_plot)